In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import swat
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [2]:
os.environ["CAS_CLIENT_SSL_CA_LIST"]="/home/ds/cascert.pem"

conn = swat.CAS("d-crd-datasci01big.dev.cyber.sas.com", 5570)

In [3]:
conn.setsessopt('WH')

+ Elapsed: 0.000342s, user: 0.000213s, sys: 8.6e-05s, mem: 0.225mb

In [4]:
out = conn.loadtable('wls_day_02_hr13.sashdat', caslib='WH')

NOTE: Cloud Analytic Services made the file wls_day_02_hr13.sashdat available as table WLS_DAY_02_HR13 in caslib CASUSER(daherr).


In [5]:
clause = "EventID = 4625"
where = clause.format(table="", not_missing="Is Not Missing")

conn.partition(table={"name": "wls_day_02_hr13", "where":where}, 
               casout={"name": "WLS_DAY02_HR13_FAILED"})

[caslib]

 'CASUSER(daherr)'

[tableName]

 'WLS_DAY02_HR13_FAILED'

[rowsTransferred]

 0

[shuffleWaitTime]

 0.0

[minShuffleWaitTime]

 1e+300

[maxShuffleWaitTime]

 0.0

[averageShuffleWaitTime]

 0.0

[casTable]

 CASTable('WLS_DAY02_HR13_FAILED', caslib='CASUSER(daherr)')

+ Elapsed: 0.166s, user: 0.104s, sys: 0.0606s, mem: 3.8mb

In [6]:
failedFull = conn.CASTable('WLS_DAY02_HR13_FAILED')

In [7]:
len(failedFull)

1507

In [8]:
failedFull.UserName.distinct()

,Column,NDistinct,NMiss,Trunc
0,UserName,142.0,0.0,0.0


In [9]:
conn.loadactionset("fedsql")

NOTE: Added action set 'fedsql'.


[actionset]

 'fedsql'

+ Elapsed: 0.00316s, user: 0.00205s, sys: 0.00105s, mem: 0.262mb

In [10]:
query = f"CREATE TABLE WLS_DAY02_HR13_FAILED_USERS AS SELECT UserName, count(UserName) as Failures \
          FROM WLS_DAY02_HR13_FAILED GROUP BY UserName"

conn.fedsql.execdirect(query)

NOTE: Table WLS_DAY02_HR13_FAILED_USERS was created in caslib CASUSER(daherr) with 142 rows returned.


+ Elapsed: 0.746s, user: 0.977s, sys: 2.19s, mem: 380mb

In [11]:
failedUsers = conn.CASTable('WLS_DAY02_HR13_FAILED_USERS').sort('Failures', ascending=False)

In [13]:
failedUsers.head(10)

,UserName,FAILURES
0,Comp065845$,636
1,Administrator,207
2,AppService,163
3,User031784,79
4,User006226,36
5,Scanner,33
6,User759472,19
7,User986685,18
8,User718489,17
9,User096590,15


In [ ]:
failedUsers.FAILURES.summary()['Summary'].transpose()

In [ ]:
y_pos = np.arange(20)

plt.figure(figsize=(20,5))
plt.bar(y_pos, failedUsers.FAILURES.values[:20], align='center', alpha=0.5)
plt.xticks(y_pos, failedUsers.UserName.values[:20], rotation=45)
plt.ylabel('Failed Login Attempts')
plt.title('Top 20 Login Failures By Username')
plt.savefig('loginFailures.png', bbox_inches='tight')

plt.show()

In [ ]:
conn.close()